### **CONECTANDO O GOOGLE SHEETS COM O PYTHON**

In [ ]:
from __future__ import print_function

import os.path
import pandas as pd
import numpy as np
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

nome_mes = 'FEVEREIRO'
linhas = 1346

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = '1_TzimHTZEGRFQH1r58YNj75FFr15Rw4Dh8U_gYsAy1Y'
SAMPLE_RANGE_NAME = f'Relatório - {nome_mes} 24!A1:AS{linhas}'



def main():
    creds = None

    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        service = build('sheets', 'v4', credentials=creds)

        # Call the Sheets API
        sheet = service.spreadsheets()
        result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                    range=SAMPLE_RANGE_NAME).execute()
        
        valores = result['values']
        # print(valores)
        return valores

    except HttpError as err:
        print(err)


if __name__ == '__main__':
    main()

valores = main()
print(valores)

### **CRIANDO A TABELA**

In [ ]:
espaco = ' '
numero_semanas = 4
semana_resultado = 1
nome_arquivo_final = f'D:\Perfil\Desktop\PROJETOS EMPRESA\PEDRO LEVI\SERVIÇO DE REPOSIÇÃO\ARQUIVOS PRONTOS\\RESULTADO GERAL SEMANAL e MENSAL\\CONSOLIDADO ATÉ SEMANA {semana_resultado} - {nome_mes}.xlsx'

semanas = {
    'semana1': 'SEMANA 1             (05/02 A 10/02)',
    'semana2': 'SEMANA 2             (12/01 A 17/02)',
    'semana3': 'SEMANA 3             (19/02 A 24/02)',
    'semana4': 'SEMANA 4             (26/02 A 02/03)',
    'semana5': 'SEMANA 5             (XX/XX A XX/XX)'
}


df_regioes = pd.read_excel('D:\\Perfil\\Desktop\\PROJETOS EMPRESA\\PEDRO LEVI\\SERVIÇO DE REPOSIÇÃO\\ARQUIVOS BASE\\REGIOES.xlsx')

df_visitas = {}
colunas_selecionadas = [
            'REDE',
            'LOJA',
            'REPOSITOR',
            'FREQUÊNCIA',
            'SEMANA 1',
            'SEMANA 2',
            'SEMANA 3',
            'SEMANA 4',
            'SEMANA 5'
        ]

for posi, linha in enumerate(valores):
    if posi == 0:
        for valor_coluna in linha:
            if valor_coluna in colunas_selecionadas:
                df_visitas[valor_coluna] = []
    else:
        for posi2, valor_linha in enumerate(linha):
            if valores[0][posi2] in colunas_selecionadas:
                df_visitas[valores[0][posi2]].append(valor_linha)
    
df_visitas = pd.DataFrame(df_visitas)


df_visitas.dropna(how='any', inplace=True)
df_visitas.reset_index(inplace=True, drop=True)
df_visitas['REPOSITOR'] = df_visitas['REPOSITOR'].str.strip()

df_visitas = pd.merge(df_visitas, df_regioes, on = 'REPOSITOR')
df_visitas = df_visitas[[
    'REDE', 'REGIÃO', 
    'LOJA', 'REPOSITOR', 
    'FREQUÊNCIA', 'SEMANA 1', 
    'SEMANA 2', 'SEMANA 3', 
    'SEMANA 4', 'SEMANA 5'
]]

df_visitas['FREQUÊNCIA'] = df_visitas['FREQUÊNCIA'].str.removesuffix(',00')
df_visitas['FREQUÊNCIA'] = pd.to_numeric(df_visitas['FREQUÊNCIA'])
df_visitas['SEMANA 1'] = pd.to_numeric(df_visitas['SEMANA 1'])
df_visitas['SEMANA 2'] = pd.to_numeric(df_visitas['SEMANA 2'])
df_visitas['SEMANA 3'] = pd.to_numeric(df_visitas['SEMANA 3'])
df_visitas['SEMANA 4'] = pd.to_numeric(df_visitas['SEMANA 4'])
df_visitas['SEMANA 5'] = pd.to_numeric(df_visitas['SEMANA 5'])


# VISUALIZANDO AS TABELAS
display(df_visitas.head())
# display(df_regioes.head(4))


# VISUALIZANDO AS INFORMAÇÕES DAS TABELAS
print(df_visitas.info())
# print(df_visitas['REPOSITOR'].unique())

### RESUMINDO INFORMAÇÕES

**REPOSITOR | REGIÃO | LOJAS BASE**

In [ ]:
df_repositor_rg_lj_base = pd.DataFrame(df_visitas[['REPOSITOR', 'REGIÃO']].value_counts())
df_repositor_rg_lj_base.reset_index(inplace=True)
df_repositor_rg_lj_base.rename(columns={'count': 'LOJAS BASE'}, inplace=True)

display(df_repositor_rg_lj_base.head(4))

**CONTRATADO MÊS | VISITAS REAL MÊS | % MÊS**

In [ ]:
df_contratado_mes = df_visitas[['REPOSITOR', 'FREQUÊNCIA']].groupby(by=['REPOSITOR']).sum()
df_contratado_mes.reset_index(inplace=True)
df_contratado_mes.rename(columns={'FREQUÊNCIA': 'CONTRATADO MÊS'}, inplace=True)
df_contratado_mes['CONTRATADO MÊS'] = df_contratado_mes['CONTRATADO MÊS'] * numero_semanas
df_contratado_mes.sort_values(by='REPOSITOR', inplace=True)

display(df_contratado_mes.head(4))

In [ ]:
df_visitas_real_mes = df_visitas[['REPOSITOR', 'SEMANA 1', 'SEMANA 2', 'SEMANA 3', 'SEMANA 4', 'SEMANA 5']].groupby(by=['REPOSITOR']).sum().reset_index()
df_visitas_real_mes['REAL MÊS'] = (
    df_visitas_real_mes['SEMANA 1'] + 
    df_visitas_real_mes['SEMANA 2'] + 
    df_visitas_real_mes['SEMANA 3'] + 
    df_visitas_real_mes['SEMANA 4'] + 
    df_visitas_real_mes['SEMANA 5']
)
df_visitas_real_mes = df_visitas_real_mes[['REPOSITOR', 'REAL MÊS']]
df_visitas_real_mes.sort_values(by='REPOSITOR', inplace=True)

display(df_visitas_real_mes.head(3))

In [ ]:
percentagem_mes = pd.DataFrame(df_visitas_real_mes['REPOSITOR'])

percentagem_mes['% \n MÊS'] = round((df_visitas_real_mes['REAL MÊS']  / df_contratado_mes['CONTRATADO MÊS']) * 100)

display(percentagem_mes.head())

**JUNTANDO: REPOSITOR | REGIÃO | LOJAS BASE + CONTRATADO MÊS | VISITAS REAL MÊS | % MÊS**

In [ ]:
df_dados_mes = pd.merge(df_repositor_rg_lj_base, df_contratado_mes, on = 'REPOSITOR')
df_dados_mes = pd.merge(df_dados_mes, df_visitas_real_mes, on = 'REPOSITOR')
df_dados_mes = pd.merge(df_dados_mes, percentagem_mes, on = 'REPOSITOR')
display(df_dados_mes.head(3))

### DADOS SEMANAS

In [ ]:
df_contratado_semanal = pd.DataFrame({
    'REPOSITOR': df_visitas[['REPOSITOR', 'FREQUÊNCIA']].groupby(by=['REPOSITOR']).sum().reset_index()['REPOSITOR'],
    'FREQUÊNCIA P/ SEMANA': df_visitas[['REPOSITOR', 'FREQUÊNCIA']].groupby(by=['REPOSITOR']).sum().reset_index()['FREQUÊNCIA']
})


display(df_contratado_semanal.head(3))

### SEMANA 1

In [ ]:
# CONTRATADO SEMANA

df_semana1 = pd.DataFrame({
    'REPOSITOR': df_contratado_semanal['REPOSITOR'],
    espaco: np.nan,
    espaco + espaco: np.nan,
    f"{semanas['semana1']} CONTRATADO":  df_contratado_semanal['FREQUÊNCIA P/ SEMANA']
})

espaco += espaco + espaco + espaco

display(df_semana1.head(3))

In [ ]:
# VISITAS REAL SEMANA

df_semana1 = pd.merge(df_semana1, df_visitas[[
    'REPOSITOR', 'SEMANA 1', 
    'SEMANA 2', 'SEMANA 3', 
    'SEMANA 4', 'SEMANA 5']].groupby(by=['REPOSITOR']).sum().reset_index()[['REPOSITOR', 'SEMANA 1']], on='REPOSITOR')

display(df_semana1.head(3))

In [ ]:
# PERCENTAGEM REAL SEMANA

df_semana1[f'{semanas["semana1"]} \n %'] = round((df_semana1['SEMANA 1'] / df_semana1[f'{semanas["semana1"]} CONTRATADO']) * 100)
df_semana1[espaco] = np.nan

display(df_semana1.head(3))

### SEMANA 2

In [ ]:
# CONTRATADO SEMANA

df_semana2 = pd.DataFrame({
    'REPOSITOR': df_contratado_semanal['REPOSITOR'],
    f"{semanas['semana2']} CONTRATADO":  df_contratado_semanal['FREQUÊNCIA P/ SEMANA']
})

display(df_semana2.head(3))

In [ ]:
# VISITAS REAL SEMANA

df_semana2 = pd.merge(df_semana2, df_visitas[[
    'REPOSITOR', 'SEMANA 1', 
    'SEMANA 2', 'SEMANA 3', 
    'SEMANA 4', 'SEMANA 5']].groupby(by=['REPOSITOR']).sum().reset_index()[['REPOSITOR', 'SEMANA 2']], on='REPOSITOR')

display(df_semana2.head(3))

In [ ]:
# PERCENTAGEM REAL SEMANA
espaco += espaco

df_semana2[f'{semanas["semana2"]} \n %'] = round((df_semana2['SEMANA 2'] / df_semana2[f'{semanas["semana2"]} CONTRATADO']) * 100)
df_semana2[espaco] = np.nan

display(df_semana1.head(3))

### SEMANA 3

In [ ]:
# CONTRATADO SEMANA

df_semana3 = pd.DataFrame({
    'REPOSITOR': df_contratado_semanal['REPOSITOR'],
    f"{semanas['semana3']} CONTRATADO":  df_contratado_semanal['FREQUÊNCIA P/ SEMANA']
})

display(df_semana3.head(3))

In [ ]:
# VISITAS REAL SEMANA

df_semana3 = pd.merge(df_semana3, df_visitas[[
    'REPOSITOR', 'SEMANA 1', 
    'SEMANA 2', 'SEMANA 3', 
    'SEMANA 4', 'SEMANA 5']].groupby(by=['REPOSITOR']).sum().reset_index()[['REPOSITOR', 'SEMANA 3']], on='REPOSITOR')

display(df_semana3.head(3))

In [ ]:
# PERCENTAGEM REAL SEMANA
espaco += espaco

df_semana3[f'{semanas["semana3"]} \n %'] = round((df_semana3['SEMANA 3'] / df_semana3[f'{semanas["semana3"]} CONTRATADO']) * 100)
df_semana3[espaco] = np.nan

display(df_semana3.head(3))

### SEMANA 4

In [ ]:
# CONTRATADO SEMANA

df_semana4 = pd.DataFrame({
    'REPOSITOR': df_contratado_semanal['REPOSITOR'],
    f"{semanas['semana4']} CONTRATADO":  df_contratado_semanal['FREQUÊNCIA P/ SEMANA']
})

display(df_semana4.head(3))

In [ ]:
# VISITAS REAL SEMANA

df_semana4 = pd.merge(df_semana4, df_visitas[[
    'REPOSITOR', 'SEMANA 1', 
    'SEMANA 2', 'SEMANA 3', 
    'SEMANA 4', 'SEMANA 5']].groupby(by=['REPOSITOR']).sum().reset_index()[['REPOSITOR', 'SEMANA 4']], on='REPOSITOR')

display(df_semana4.head(3))

In [ ]:
# PERCENTAGEM REAL SEMANA
espaco += espaco

df_semana4[f'{semanas["semana4"]} \n %'] = round((df_semana4['SEMANA 4'] / df_semana4[f'{semanas["semana4"]} CONTRATADO']) * 100)
df_semana4[espaco] = np.nan

display(df_semana4.head(3))

### SEMANA 5

In [ ]:
# CONTRATADO SEMANA

df_semana5 = pd.DataFrame({
    'REPOSITOR': df_contratado_semanal['REPOSITOR'],
    f"{semanas['semana5']} CONTRATADO":  df_contratado_semanal['FREQUÊNCIA P/ SEMANA']
})

display(df_semana5.head(3))

In [ ]:
# VISITAS REAL SEMANA

df_semana5 = pd.merge(df_semana5, df_visitas[[
    'REPOSITOR', 'SEMANA 1', 
    'SEMANA 2', 'SEMANA 3', 
    'SEMANA 4', 'SEMANA 5']].groupby(by=['REPOSITOR']).sum().reset_index()[['REPOSITOR', 'SEMANA 5']], on='REPOSITOR')

display(df_semana5.head(3))

In [ ]:
# PERCENTAGEM REAL SEMANA

df_semana5[f'{semanas["semana5"]} \n %'] = round((df_semana5['SEMANA 5'] / df_semana5[f'{semanas["semana5"]} CONTRATADO']) * 100)

display(df_semana5.head(3))

### **JUNTANDO TODAS AS INFORMAÇÕES**

In [ ]:
df_final = pd.merge(df_dados_mes, df_semana1, on='REPOSITOR')
df_final = pd.merge(df_final, df_semana2, on='REPOSITOR')
df_final = pd.merge(df_final, df_semana3, on='REPOSITOR')
df_final = pd.merge(df_final, df_semana4, on='REPOSITOR')
df_final = pd.merge(df_final, df_semana5, on='REPOSITOR')

df_final.sort_values(by='% \n MÊS', ascending=False, inplace=True)
df_final[[
    '% \n MÊS', 
    f'{semanas["semana1"]} \n %',
    f'{semanas["semana2"]} \n %',
    f'{semanas["semana3"]} \n %',
    f'{semanas["semana4"]} \n %',
    f'{semanas["semana5"]} \n %'
]] = df_final[[
    '% \n MÊS', 
    f'{semanas["semana1"]} \n %',
    f'{semanas["semana2"]} \n %',
    f'{semanas["semana3"]} \n %',
    f'{semanas["semana4"]} \n %',
    f'{semanas["semana5"]} \n %'
]].astype(str) + '%'


display(df_final.head(2))

**EXPORTANDO DADOS**

In [ ]:
df_final.to_excel(nome_arquivo_final, index=False)

os.startfile(nome_arquivo_final)
os.startfile('D:\\Perfil\\Desktop\\PROJETOS EMPRESA\\PEDRO LEVI\\SERVIÇO DE REPOSIÇÃO\\ARQUIVOS COMPLEMENTARES\\FORMATAÇÃO PRONTA.xlsx')